# Recommendation: Code example in python3

### Collecting Preferences

In [1]:
# A dictionary of movie critics and their ratings of a small set of movies

critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5,
'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0,
'You, Me and Dupree': 3.5},
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
'The Night Listener': 4.5, 'Superman Returns': 4.0,
'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
'You, Me and Dupree': 2.0},
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

In [2]:
critics['Lisa Rose']['Lady in the Water']

2.5

In [4]:
critics['Toby']['Snakes on a Plane']

4.5

In [5]:
critics['Toby']

{'Snakes on a Plane': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 1.0}

# Finding similar users

In [7]:
# Euclidean distance score

from math import sqrt

# Return a distance-based similarity score for person 1 and person 2

def sim_distance(prefs, person1, person2):
    si = dict()
    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item]=1

    if len(si)==0: return 0

    # Add up the squares of all the differences
    sum_of_squares = sum([pow(prefs[person1][item]-prefs[person2][item],2) for item in prefs[person1] if item in prefs[person2]])
    return (1/(1+sum_of_squares))

sim_distance(critics, 'Lisa Rose', 'Gene Seymour')

0.14814814814814814

In [8]:
"""Pearson correlation score - works better where the data isn't well normalized 
e.g. if critics are routinely more harsh than average. Pearson correlation score corrects for grade inflation whereas Euclidean score will say two critics are dissimilar
because one is consistently harsher than the other, even if their tastes are very similar. 
Pearson correlation coefficient - how much variables change together divided by the product of how much they vary individually"""

# Returns the Pearson correlation coefficient for person 1 and person 2

def sim_pearson(prefs, p1, p2):
    si = dict()
    for item in prefs[p1]:
        if item in prefs[p2]:
            si[item]=1
            
    if len(si) == 0: return 0
    
    sum1 = sum([prefs[p1][it] for it in si])
    sum2 = sum([prefs[p2][it] for it in si])
    
    sum1Sq = sum([pow(prefs[p1][it],2) for it in si])
    sum2Sq = sum([pow(prefs[p2][it],2) for it in si])
    
    pSum = sum([prefs[p1][it]*prefs[p2][it] for it in si])
    
    num = pSum - ((sum1*sum2)/len(si))
    den = sqrt((sum1Sq - pow(sum1,2)/len(si))*(sum2Sq-pow(sum2,2)/len(si)))
    if den == 0: return 0
    
    I = num/den
    
    return I

sim_pearson(critics, 'Lisa Rose', 'Gene Seymour')

0.39605901719066977

## Ranking the critics 

In [9]:

# To get an ordered list of people with similar tastes to the specified person

def topmatches(prefs, person, n=5, similarity = sim_pearson):
    scores = [(similarity(prefs, person, other), other) for other in prefs if other!=person]
    
    scores.sort()
    scores.reverse()
    return scores[0:n]

topmatches(critics, 'Toby', n=3)

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig')]

In [10]:
# Get recommendations for a person by using a weighted average of every other user's rankings

def getrecommendations(prefs, person, similarity = sim_pearson):
    totals = dict()
    simSums = dict()
    
    for other in prefs:
        if other==person: continue
        sim = similarity(prefs, person, other)
        
        if sim<=0: continue
        for item in prefs[other]:
            if item not in prefs[person] or prefs[person][item]==0:
                totals.setdefault(item,0)
                totals[item]+=prefs[other][item]*sim
                
                simSums.setdefault(item,0)
                simSums[item]+=sim
                
    rankings = [(total/simSums[item],item) for item,total in totals.items()]
    
    rankings.sort()
    rankings.reverse()
    
    return rankings

getrecommendations(critics, 'Toby')

[(3.3477895267131013, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.5309807037655645, 'Just My Luck')]

In [11]:
def transformprefs(prefs):
    result=dict()
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item,{})
            result[item][person] = prefs[person][item]
        
    return result

movies = transformprefs(critics)
topmatches(movies, 'Superman Returns')
getrecommendations(movies, 'Just My Luck')

[(4.0, 'Michael Phillips'), (3.0, 'Jack Matthews')]

In [12]:
def transformprefs(prefs):
    result=dict()
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item,{})
            result[item][person] = prefs[person][item]
        
    return result

movies = transformprefs(critics)
topmatches(movies, 'Superman Returns')
getrecommendations(movies, 'Just My Luck')

[(4.0, 'Michael Phillips'), (3.0, 'Jack Matthews')]

In [13]:
#Use transformprefs to give a list of items with their user rating, then loop over every item through topmatches to get most 
# similar items along with their similarity scores
# Item-based filtering outperforms user-based filtering in sparse datasets, perform equally in dense datasets. Item-based filtering
# is significantly faster on a large dataset
def calculateSimilarItems(prefs, n=10):
    result = dict()
    itemPrefs = transformprefs(prefs)
    c=0
    for item in itemPrefs:
        c+=1
        if c%100==0: print(c, len(itemPrefs))
        scores = topmatches(itemPrefs, item, n=n, similarity = sim_distance)
        result[item] = scores
    return result

itemsim = calculateSimilarItems(critics)
print(itemsim)

{'The Night Listener': [(0.2857142857142857, 'Lady in the Water'), (0.18181818181818182, 'Snakes on a Plane'), (0.15384615384615385, 'Just My Luck'), (0.14814814814814814, 'You, Me and Dupree'), (0.10256410256410256, 'Superman Returns')], 'Lady in the Water': [(0.4, 'You, Me and Dupree'), (0.2857142857142857, 'The Night Listener'), (0.2222222222222222, 'Snakes on a Plane'), (0.2222222222222222, 'Just My Luck'), (0.09090909090909091, 'Superman Returns')], 'Snakes on a Plane': [(0.2222222222222222, 'Lady in the Water'), (0.18181818181818182, 'The Night Listener'), (0.16666666666666666, 'Superman Returns'), (0.10526315789473684, 'Just My Luck'), (0.05128205128205128, 'You, Me and Dupree')], 'Just My Luck': [(0.2222222222222222, 'Lady in the Water'), (0.18181818181818182, 'You, Me and Dupree'), (0.15384615384615385, 'The Night Listener'), (0.10526315789473684, 'Snakes on a Plane'), (0.06451612903225806, 'Superman Returns')], 'You, Me and Dupree': [(0.4, 'Lady in the Water'), (0.18181818181